In [1]:
# IMPORT MODULES
import sys, json
sys.path.append('/Users/kimdohoon/git/spotify-data-pipeline/lib')
import spark_modules as lib_spark

In [2]:
# VARIABLES
PATH = "file:/Users/kimdohoon/git/spotify-data-pipeline/datas/JSON/playlists/Hot Hits Korea.json"

# BUILD SPARK SESSION
spark = lib_spark.build_spark_session()

23/06/26 14:46:58 WARN Utils: Your hostname, neivekim76.local resolves to a loopback address: 127.0.0.1; using 192.168.70.41 instead (on interface en0)
23/06/26 14:46:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/26 14:46:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/kimdohoon/.pyenv/versions/3.7.16/envs/airflow/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
# READ JSON
dataframe = lib_spark.read_JSON(PATH)
playlist_name = dataframe.select('name').first()[0]

# TEST
print(playlist_name)

Hot Hits Korea


In [4]:
# PARSE JSON DATAS
df_tracks = lib_spark.explode_dict(dataframe, "tracks")
df_items = lib_spark.explode_list(df_tracks, "items")
df_added = lib_spark.explode_dict(df_items, "added_by")

# TEST
df_tracks.show()
df_items.show()
df_added.show()

+--------------------+--------------------+-----+----+------+--------+-----+
|                href|               items|limit|next|offset|previous|total|
+--------------------+--------------------+-----+----+------+--------+-----+
|https://api.spoti...|[{2023-06-13T13:5...|  100|null|     0|    null|   54|
+--------------------+--------------------+-----+----+------+--------+-----+

+--------------------+--------------------+--------+-------------+--------------------+---------------+
|            added_at|            added_by|is_local|primary_color|               track|video_thumbnail|
+--------------------+--------------------+--------+-------------+--------------------+---------------+
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{album, [{{https...|       

In [5]:
df_items.createOrReplaceTempView("df_items")
df_test = spark.sql("SELECT * FROM df_items")
df_test.show()

+--------------------+--------------------+--------+-------------+--------------------+---------------+
|            added_at|            added_by|is_local|primary_color|               track|video_thumbnail|
+--------------------+--------------------+--------+-------------+--------------------+---------------+
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{album, [{{https...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{album, [{{https...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         nul

In [6]:
from pyspark.sql.functions import col
df_track = df_items.filter(~(col('track').isNull()))
df_track.show()

+--------------------+--------------------+--------+-------------+--------------------+---------------+
|            added_at|            added_by|is_local|primary_color|               track|video_thumbnail|
+--------------------+--------------------+--------+-------------+--------------------+---------------+
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{album, [{{https...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{album, [{{https...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         null|{{single, [{{http...|         {null}|
|2023-06-13T13:57:44Z|{{https://open.sp...|   false|         nul

In [10]:
df_track.printSchema()

root
 |-- added_at: string (nullable = true)
 |-- added_by: struct (nullable = true)
 |    |-- external_urls: struct (nullable = true)
 |    |    |-- spotify: string (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- uri: string (nullable = true)
 |-- is_local: boolean (nullable = true)
 |-- primary_color: string (nullable = true)
 |-- track: struct (nullable = true)
 |    |-- album: struct (nullable = true)
 |    |    |-- album_type: string (nullable = true)
 |    |    |-- artists: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string

In [8]:
df_track_fin = lib_spark.explode_test(df_track, "track")

AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "track". Need a complex type [STRUCT, ARRAY, MAP] but got "BOOLEAN".; line 1 pos 0

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "track LIKE {%" due to data type mismatch: Parameter 1 requires the "STRING" type, however "track" has the type "STRUCT<album: STRUCT<album_type: STRING, artists: ARRAY<STRUCT<external_urls: STRUCT<spotify: STRING>, href: STRING, id: STRING, name: STRING, type: STRING, uri: STRING>>, available_markets: ARRAY<STRING>, external_urls: STRUCT<spotify: STRING>, href: STRING, id: STRING, images: ARRAY<STRUCT<height: BIGINT, url: STRING, width: BIGINT>>, name: STRING, release_date: STRING, release_date_precision: STRING, total_tracks: BIGINT, type: STRING, uri: STRING>, artists: ARRAY<STRUCT<external_urls: STRUCT<spotify: STRING>, href: STRING, id: STRING, name: STRING, type: STRING, uri: STRING>>, available_markets: ARRAY<STRING>, disc_number: BIGINT, duration_ms: BIGINT, episode: BOOLEAN, explicit: BOOLEAN, external_ids: STRUCT<isrc: STRING>, external_urls: STRUCT<spotify: STRING>, href: STRING, id: STRING, is_local